<a href="https://colab.research.google.com/github/Gilf641/EVA4/blob/master/S15/OfficeDataset_S15_Preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Hardware Properties**

In [3]:
import torch
from IPython.display import Image, clear_output 
import time as t

print('PyTorch %s %s' % (torch.__version__, torch.cuda.get_device_properties(0) if torch.cuda.is_available() else 'CPU'))

PyTorch 1.5.1+cu101 _CudaDeviceProperties(name='Tesla K80', major=3, minor=7, total_memory=11441MB, multi_processor_count=13)


**Mount the Drive**

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


**Unzip the File**

In [3]:
# import zipfile
# def zip_data(zipfile, path_to_zipfile, directory_to_extract):
#   with zipfile.ZipFile(path_to_zipfile, 'r') as zip:
#     zip.extractall(directory_to_extract)
#   print("Zipfile extracted to ", directory_to_extract)

# # zipfile = "/content/drive/My Drive/DepthEstimation/TestZip.zip"

# path_to_zipfile = "/content/drive/My Drive/DepthEstimation/data_TenK.zip"
# directory_to_extract = "/content/drive/My Drive/ZipExtract"
# zip_data(zipfile, path_to_zipfile, directory_to_extract)

Zipfile extracted to  /content/drive/My Drive/ZipExtract


**Some Basic Checks**

In [5]:
import os
os.listdir("/content/drive/My Drive/ZipExtract/data_TenK")

['BG', 'BG_FG Masks', 'BG_FG', 'DepthMaps']

In [7]:
import os
len(os.listdir("/content/drive/My Drive/ZipExtract/data_TenK/DepthMaps"))

10000

**Set the path for all 4 type of Images**

In [13]:
zip_path = "/content/drive/My Drive/ZipExtract/data_TenK"
subd = os.listdir(zip_path) #Returns sub-directories

bg_path, bgfg_mask_path = os.path.join(zip_path, subd[0]), os.path.join(zip_path, subd[1])
bgfg_path, depthmap_path = os.path.join(zip_path, subd[2]), os.path.join(zip_path, subd[3])


for i in [bg_path, bgfg_mask_path, bgfg_path, depthmap_path]:
  print(i)

/content/drive/My Drive/ZipExtract/data_TenK/BG
/content/drive/My Drive/ZipExtract/data_TenK/BG_FG Masks
/content/drive/My Drive/ZipExtract/data_TenK/BG_FG
/content/drive/My Drive/ZipExtract/data_TenK/DepthMaps


**Image Segmentation File**

In [14]:
# Now the first task is to clip input and output
# This is done since BG has only 100 images while BG_FG has around 400K, but for this I'm using 10K
# Now to do this I've used this snippet below, which will clip in the inputs with it's corresponding ground truth

masks = [mask for mask in os.listdir(bgfg_mask_path)] # Mask Files
bg_idx = [int(m.split('_')[1]) for m in masks] # Background file index

bgfiles = [os.path.join(bg_path, bg) for bg in os.listdir(bg_path)] # all 100 BG Files
bg_files = [os.path.join(bgfg_mask_path, bgfiles[idx-1]) for idx in bg_idx ] # BG Files of Proper index
bg_files

['/content/drive/My Drive/ZipExtract/data_TenK/BG/bg003.jpg',
 '/content/drive/My Drive/ZipExtract/data_TenK/BG/bg003.jpg',
 '/content/drive/My Drive/ZipExtract/data_TenK/BG/bg003.jpg',
 '/content/drive/My Drive/ZipExtract/data_TenK/BG/bg003.jpg',
 '/content/drive/My Drive/ZipExtract/data_TenK/BG/bg003.jpg',
 '/content/drive/My Drive/ZipExtract/data_TenK/BG/bg003.jpg',
 '/content/drive/My Drive/ZipExtract/data_TenK/BG/bg003.jpg',
 '/content/drive/My Drive/ZipExtract/data_TenK/BG/bg003.jpg',
 '/content/drive/My Drive/ZipExtract/data_TenK/BG/bg003.jpg',
 '/content/drive/My Drive/ZipExtract/data_TenK/BG/bg003.jpg',
 '/content/drive/My Drive/ZipExtract/data_TenK/BG/bg003.jpg',
 '/content/drive/My Drive/ZipExtract/data_TenK/BG/bg003.jpg',
 '/content/drive/My Drive/ZipExtract/data_TenK/BG/bg003.jpg',
 '/content/drive/My Drive/ZipExtract/data_TenK/BG/bg003.jpg',
 '/content/drive/My Drive/ZipExtract/data_TenK/BG/bg003.jpg',
 '/content/drive/My Drive/ZipExtract/data_TenK/BG/bg003.jpg',
 '/conte

In [26]:
# bg_files = [os.path.join(bg_path, bg) for bg in os.listdir(bg_path)] # BG Files

# I have zipped both bg and bg_fg as image segmentation input, this is obvious. I don't think I need to explain here

bgfg_mask_files = [os.path.join(bgfg_mask_path, mask) for mask in os.listdir(bgfg_mask_path)] # Mask Files
bgfg_files = [os.path.join(bgfg_path, bgfg) for bgfg in os.listdir(bgfg_path)] # bgfg files
# dpmap_files = [os.path.join(depthmap_path, map) for map in os.listdir(depthmap_path)] #Depth Maps

img_seg = list(zip(bg_files, bgfg_files, bgfg_mask_files))
# # ziplist = list(zip1)
type(img_seg)
img_seg # this is the final data for image segmentations, go and use it

[('/content/drive/My Drive/ZipExtract/data_TenK/BG/bg001.jpg',
  '/content/drive/My Drive/ZipExtract/data_TenK/BG_FG/bg_001_fg_050_011.jpeg',
  '/content/drive/My Drive/ZipExtract/data_TenK/BG_FG Masks/bg_001_fg_050_011.jpeg'),
 ('/content/drive/My Drive/ZipExtract/data_TenK/BG/bg001.jpg',
  '/content/drive/My Drive/ZipExtract/data_TenK/BG_FG/bg_001_fg_050_012.jpeg',
  '/content/drive/My Drive/ZipExtract/data_TenK/BG_FG Masks/bg_001_fg_050_012.jpeg'),
 ('/content/drive/My Drive/ZipExtract/data_TenK/BG/bg001.jpg',
  '/content/drive/My Drive/ZipExtract/data_TenK/BG_FG/bg_001_fg_050_013.jpeg',
  '/content/drive/My Drive/ZipExtract/data_TenK/BG_FG Masks/bg_001_fg_050_013.jpeg'),
 ('/content/drive/My Drive/ZipExtract/data_TenK/BG/bg001.jpg',
  '/content/drive/My Drive/ZipExtract/data_TenK/BG_FG/bg_001_fg_050_014.jpeg',
  '/content/drive/My Drive/ZipExtract/data_TenK/BG_FG Masks/bg_001_fg_050_014.jpeg'),
 ('/content/drive/My Drive/ZipExtract/data_TenK/BG/bg001.jpg',
  '/content/drive/My Driv

**Depth Estimation File**

In [ ]:
bgfg_files = [os.path.join(bgfg_path, bgfg) for bgfg in os.listdir(bgfg_path)]
dpmap_files = [os.path.join(depthmap_path, map) for map in os.listdir(depthmap_path)]

dp_est = list(zip(bgfg_files,dpmap_files))
# # # ziplist = list(zip1)
type(zip1)

list

**Check the ZIP File**

In [ ]:
a,b,c=zip(*img_seg)

In [ ]:
a

**Custom Depth Map Dataset Class**

In [27]:
from torch.utils.data import Dataset, random_split
from PIL import Image
import numpy as np
import torch
import os
import torchvision.transforms as transforms
from tqdm import notebook
import zipfile

print("Libraries Imported...")

overall_start_time = t.time()


class DepthMapDataset(Dataset):
  
  # Image Segmentation Part
  def __init__(self, data, bg_transforms, bgfg_transforms, mask_transforms):
    self.bg_images, self.bgfg_images, self.mask_images = zip(*data) # do the unzipping part
    self.bg_transforms = bg_transforms
    self.bgfg_transforms = bgfg_transforms
    self.mask_transforms = mask_transforms


  def __len__(self):
    """
    returns len of the dataset
    """
    return len(self.bgfg_images)


  def __getitem__(self, idx):
    """
    returns image data & target for the corresponding index
    """
    try:
      bg_image = Image.open(self.bg_images[idx]) # BG Images
      bgfg_image = Image.open(self.bgfg_images[idx]) # BG_FG Images
      mask_image = Image.open(self.mask_images[idx]) # Mask Images
      # depth_map = Image.open(self.depth_maps[idx]) # Depth Images

      start_time = t.time()
      bg_image = self.bg_image
      bgfg_image = self.bgfg_transforms(bgfg_image)
      
      # depth_map = self.depthmap_transforms(depth_map)



      print("Image Transformed")
      # print(bgfg_image.mode)
      # print(depth_map.mode)
      # print("Took {0} seconds.".format(round(t.time()-start_time, 4)))
      return {"bg": bg_image, "bgfg": bgfg_image, "mask_image": mask_image} #"depthmap": depth_map} # dict way of returning Depth Map
      re


    except Exception as e:
      print("Image {0} skipped due to {1}".format(self.bgfg_images[idx],e)) # this is only if some images can't be identified by PIL

    


Libraries Imported...


**Albumentation Script**

In [ ]:
from torchvision import transforms
import albumentations as A
import albumentations.pytorch as AP
import random
import numpy as np

class AlbumentationTransforms:
  """
  Helper class to create test and train transforms using Albumentations
  """
  def __init__(self, transforms_list=[]):
    transforms_list.append(AP.ToTensor()) # Transform the normalized image to Tensor
    self.transforms = A.Compose(transforms_list)


  def __call__(self, img):
    img = np.array(img) 
    return self.transforms(image=img)['image']

  

**Image Transformations**

In [ ]:
import albumentations as A
import albumentations.pytorch as AP
import cv2

# FG_BG Images
fgbg_mean = (0.513, 0.536, 0.5546)
fgbg_stdev = (0.2355, 0.2317, 0.2326)

# Depth Maps
dpmap_mean = (0.3045)
dpmap_std = (0.1130)

# Used same type of transformations for both Train & Test Data
bgfg_transf = AlbumentationTransforms([A.Normalize(mean=fgbg_mean,
                                              std=fgbg_stdev)])

dpmap_transf = AlbumentationTransforms([A.Normalize(mean=dpmap_mean,
                                                    std=dpmap_std)])


# dpmap_transforms = AlbumentationTransforms([A.Normalize(mean=dpmap_mean, 
                                              # std=dpmap_std)])
# train_dataset , test_dataset = DepthMapDataset(splitRatio = 70, test_transforms = test_transform,train_transforms = train_transform)


In [ ]:
# class TransformData(Dataset):
#     """
#     Helper Class for transforming the images using albumentations.
#     """
#     def __init__(self, data, bgfg_transform=None, dpmap_transform=None):
#         """
#         data: Train or Validation Dataset
#         transform : List of Transforms that one wants to apply
#         """
#         if not isinstance(data, zip):
#           self.bgfg_images, self.depth_maps = zip(*data)# do the unzipping part
#           self.bgfg_transform = bgfg_transform
#           self.dpmap_transform = dpmap_transform


#     def __getitem__(self, index):
#       bgfg_image = Image.open(self.bgfg_images[idx])
#       depth_map = Image.open(self.depth_maps[idx])

#       if self.transform:
#         bgfg_image = self.transform(bgfg_image)
#         # return { "bgfg": bgfg_image, "depthmap": depth_map} # do it the dataset way

#         depth_map = self.transform(depth_map)
#         return bgfg_image, depth_map

#     def __len__(self):
#         return len(self.bgfg_images)

In [ ]:
# def depth_estimation(split_ratio=70):
#   print("Creating DMD Object...")
#   dmd = DepthMapDataset(zip1) # dmd:-DepthMapDataset
#   totalLen = len(dmd)
#   print(totalLen) 

#   print("Splitting Data...")
#   traindata_len = totalLen*split_ratio//100 
#   testdata_len = totalLen - traindata_len 
  
#   train, val = random_split(dmd, [traindata_len, testdata_len]) 
#   print('split train', train)
#   print('split test', val)

#   # apply transformations
#   # print(type(train))
#   print("Applying Transformations...")

#   train_dataset = TransformData(train, bgfg_transf, dpmap_transf)
#   print("Train Data Transformed...")

#   test_dataset = TransformData(val, bgfg_transf, dpmap_transf)
#   print("Test Data Transformed...")

#   return train_dataset, test_dataset

**DataLoaders**

In [ ]:
!python -c 'import PIL; print( PIL.__version__)'

7.0.0


**Dataloaders**

In [ ]:
# Both class and func here do the same job. I'm using function for TinyImageNet.

import torch

class DataLoaders:
  def __init__(self, 
              batch_size=512,
              shuffle=True,
              num_workers=4,
              pin_memory=True,
              seed=1):
    """
    Arguments:-
    batch_size: Number of images to be passed in each batch
    shuffle(boolean):  If True, then shuffling of the dataset takes place
    num_workers(int): Number of processes that generate batches in parallel
    pin_memory(boolean):
    seed: Random Number, this is to maintain the consistency
    """
    use_cuda = torch.cuda.is_available()
    device = torch.device('cuda' if use_cuda else 'cpu')  # set device to cuda

    if use_cuda:
      torch.manual_seed(seed)
    
    self.dataLoader_args = dict(batch_size=batch_size, shuffle=True, num_workers=4, pin_memory=True ) if use_cuda else dict(batch_size=1, shuffle=True, num_workers = 1, pin_memory = True)


  def dataLoader(self, data):
    return torch.utils.data.DataLoader(data,**self.dataLoader_args)


import torch
import torchvision


def Data_To_Dataloader(trainset,testset,seed=1,batch_size=128,num_workers=4,pin_memory=True):
	"""
	Conv DataSet Obj to DataLoader
	"""

	SEED = 1

	# CUDA?
	cuda = torch.cuda.is_available()

	# For reproducibility
	torch.manual_seed(SEED)

	if cuda:
			torch.cuda.manual_seed(SEED)

	dataloader_args = dict(shuffle=True, batch_size=batch_size, num_workers=num_workers, pin_memory=pin_memory) if cuda else dict(shuffle=True, batch_size=64)

	trainloader = torch.utils.data.DataLoader(trainset, **dataloader_args)
	testloader = torch.utils.data.DataLoader(testset, **dataloader_args)


	return  trainloader, testloader

In [ ]:
trainLoader, testLoader = Data_To_Dataloader(train,test,batch_size=128)

In [ ]:
a,b = iter(trainLoader).next()

'depthmap'

In [ ]:

import torch
import torch.nn as nn
import torch.nn.functional as F


class BasicBlock(nn.Module):
    expansion = 1

    def __init__(self, in_planes, planes, stride=1):
        super(BasicBlock, self).__init__()
        self.conv1 = nn.Conv2d(
            in_planes, planes, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(planes)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3,
                               stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(planes)

        self.shortcut = nn.Sequential() # if no change in number of channels then self.shortcut is like empty
        if stride != 1 or in_planes != self.expansion*planes: # this is like if there's change in channel number only then it's used I guess
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_planes, self.expansion*planes,
                          kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(self.expansion*planes)
            )

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        out += self.shortcut(x)
        out = F.relu(out)
        return out


class Bottleneck(nn.Module):
    expansion = 4

    def __init__(self, in_planes, planes, stride=1):
        super(Bottleneck, self).__init__()
        self.conv1 = nn.Conv2d(in_planes, planes, kernel_size=1, bias=False)
        self.bn1 = nn.BatchNorm2d(planes)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3,
                               stride=stride, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(planes)
        self.conv3 = nn.Conv2d(planes, self.expansion *
                               planes, kernel_size=1, bias=False)
        self.bn3 = nn.BatchNorm2d(self.expansion*planes)

        self.shortcut = nn.Sequential()
        if stride != 1 or in_planes != self.expansion*planes:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_planes, self.expansion*planes,
                          kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(self.expansion*planes)
            )

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = F.relu(self.bn2(self.conv2(out)))
        out = self.bn3(self.conv3(out))
        out += self.shortcut(x)
        out = F.relu(out)
        return out


# This commenting is for ResNet18
class ResNet(nn.Module):
    def __init__(self, block, num_blocks, num_classes=200): # block = BasicBlock, num_blocks = [2,2,2,2]
        super(ResNet, self).__init__()
        self.in_planes = 64  # maybe this is the number of kernels at the start

        self.conv1 = nn.Conv2d(3, 64, kernel_size=3,
                               stride=1, padding=1, bias=False) # conv2 with incoming 3 channels
        self.bn1 = nn.BatchNorm2d(64)
        self.layer1 = self._make_layer(block, 64, num_blocks[0], stride=1)
        self.layer2 = self._make_layer(block, 128, num_blocks[1], stride=2)
        self.layer3 = self._make_layer(block, 256, num_blocks[2], stride=2)
        self.layer4 = self._make_layer(block, 512, num_blocks[3], stride=2)
        self.linear = nn.Linear(512*block.expansion, num_classes)

    def _make_layer(self, block, planes, num_blocks, stride):
        strides = [stride] + [1]*(num_blocks-1)
        layers = []
        for stride in strides:
            layers.append(block(self.in_planes, planes, stride))
            self.in_planes = planes * block.expansion
        return nn.Sequential(*layers)

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        out = F.avg_pool2d(out, 28)
        out = out.view(out.size(0), -1)
        out = self.linear(out)
        return out


def ResNet18():
    return ResNet(BasicBlock, [2, 2, 2, 2])


def ResNet34():
    return ResNet(BasicBlock, [3, 4, 6, 3])


def ResNet50():
    return ResNet(Bottleneck, [3, 4, 6, 3])


def ResNet101():
    return ResNet(Bottleneck, [3, 4, 23, 3])


def ResNet152():
    return ResNet(Bottleneck, [3, 8, 36, 3])


def test():
    net = ResNet18()
    y = net(torch.randn(1, 3, 32, 32))
    print(y.size())


In [ ]:
from torchsummary import summary
import torch
cuda = torch.cuda.is_available()
device = torch.device("cuda" if cuda else "cpu")
model = ResNet34().to(device)
summary(model, input_size=(3, 224, 224))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 224, 224]           1,728
       BatchNorm2d-2         [-1, 64, 224, 224]             128
            Conv2d-3         [-1, 64, 224, 224]          36,864
       BatchNorm2d-4         [-1, 64, 224, 224]             128
            Conv2d-5         [-1, 64, 224, 224]          36,864
       BatchNorm2d-6         [-1, 64, 224, 224]             128
        BasicBlock-7         [-1, 64, 224, 224]               0
            Conv2d-8         [-1, 64, 224, 224]          36,864
       BatchNorm2d-9         [-1, 64, 224, 224]             128
           Conv2d-10         [-1, 64, 224, 224]          36,864
      BatchNorm2d-11         [-1, 64, 224, 224]             128
       BasicBlock-12         [-1, 64, 224, 224]               0
           Conv2d-13         [-1, 64, 224, 224]          36,864
      BatchNorm2d-14         [-1, 64, 2

In [ ]:
import torch.optim as optim
import torch.nn as nn
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=1e-4, weight_decay=1e-2)


In [ ]:
from tqdm import tqdm
import numpy as np
import torch.nn.functional as F
import torch


class Test_Train():
  def __init__(self, model, device, optimizer, criterion, scheduler=True ):


    self.model = model
    self.device = device
    self.optimizer = optimizer
    self.criterion = criterion
    self.scheduler = scheduler

# # This is to hold all the values and plot some graphs to extract few good insights.
    self.train_losses = []
    self.test_losses = []
    self.train_acc = []
    self.test_acc = []
    self.train_epoch_end = []
    self.test_loss_min = np.inf # setting it to infinity(max value)
    # when the test loss becomes min I will save the particular model


  def train(self, trainloader, epoch):
    self.model.train()    # prepare model for training
    pbar = tqdm(trainloader)
    correct = 0
    processed = 0
    loss = 0
    for batch_idx, (data, target) in enumerate(pbar): # passing on data & target values to device
      data, target = data.to(self.device), target.to(self.device)
      self.optimizer.zero_grad()    # clear the gradients of all optimized variables

      # Predict
      y_pred = self.model(data)   # forward pass

      # Calculate loss
      loss = self.criterion(y_pred, target)

      # #Implementing L1 Regularization
      # if L1lambda:
      #   with torch.enable_grad():
      #     l1_loss = 0.
      #     for param in self.model.parameters():
      #       l1_loss += torch.sum(param.abs())
      #     loss = c_loss +  (L1lambda * l1_loss)



      # Backpropagation
      loss.backward()   # backward pass: compute gradient of the loss with respect to model parameters
      self.optimizer.step()   # perform a single optimization step (parameter update)
      # if self.scheduler:
      #   scheduler.step(loss)


      # Update pbar-tqdm
      pred = y_pred.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
      correct += pred.eq(target.view_as(pred)).sum().item()
      processed += len(data)

      pbar.set_description(desc= f'Loss={loss.item()} Batch_id={batch_idx} Accuracy={100*correct/processed:0.2f}')
    self.train_acc.append(100*correct/processed)
    
    self.train_epoch_end.append(self.train_acc[-1])
    self.train_losses.append(loss)


  def test(self, testloader, filename, correct_samples, correctLabels, incorrect_samples):
      self.model.eval()  # prep model for evaluation
      test_loss = 0
      correct = 0

      with torch.no_grad(): # setting gradients back to zero
          for data, target in testloader:

            img_batch = data # this is done to store data
            data, target = data.to(self.device), target.to(self.device)

            # forward pass: compute predicted outputs by passing inputs to the model
            output = self.model(data)

            # sum up batch loss
            # test_loss += F.nll_loss(output, target, reduction='sum').item()
            test_loss = self.criterion(output, target).item()


            # get the index of the max log-probability
            pred = output.argmax(dim=1, keepdim=True)
            correct += pred.eq(target.view_as(pred)).sum().item()

            # storing the entire result data as binary
            result = pred.eq(target.view_as(pred))
            # scheduler.step()

            # This is to extract incorrect samples/misclassified images
            if len(incorrect_samples) < 25:
              for i in range(0, testloader.batch_size):
                if not list(result)[i]:
                  incorrect_samples.append({'prediction': list(pred)[i], 'label': list(target.view_as(pred))[i],'image': img_batch[i]})


            # this is to extract correct samples/classified images
            if len(correct_samples) < 25:
              for i in range(0, testloader.batch_size):
                if list(result)[i]:
                  correct_samples.append({'prediction': list(pred)[i], 'label': list(target.view_as(pred))[i],'image': img_batch[i]})
                  correctLabels.append(list(target.view_as(pred))[i]) # this is for gradcam




      # save model if validation loss has decreased
      if test_loss <= self.test_loss_min:
          print('Validation loss has  decreased ({:.4f} --> {:.4f}).  Saving model ...'.format(self.test_loss_min,test_loss ))
          torch.save(self.model.state_dict(), filename)
          self.test_loss_min = test_loss


      print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.2f}%)\n'.format(
          test_loss, correct, len(testloader.dataset),
          100. * correct / len(testloader.dataset)))


      self.test_acc.append(100. * correct / len(testloader.dataset))
      self.test_losses.append(test_loss)



def cross_entropy_loss():
    """Returns:
        Cross entroy loss function
    """
    return torch.nn.CrossEntropyLoss()

def model_summary(model, input_size=(3,32,32)):
  """
  Returns Summary of the model passed in as model
  """
  return summary(model, input_size)



In [ ]:

model = ResNet18().to(device)
EPOCHS = 50
import torch.optim as optim
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr = 0.01, momentum=0.95, nesterov = True) 
scheduler = torch.optim.lr_scheduler.OneCycleLR(optimizer, max_lr = 0.01, total_steps=31000, epochs = 50, steps_per_epoch=len(trainLoader),
                                                pct_start=0.5, anneal_strategy='linear', cycle_momentum=True, 
                                                base_momentum=0.85, max_momentum=0.95, div_factor=10, final_div_factor=1)


model1 = execute.Test_Train(model, device, optimizer, criterion, scheduler=True)



for epoch in range(1, EPOCHS+1):
  print('EPOCH: ',epoch)
  model1.train(trainLoader, epoch)
  scheduler.step()
  model1.test(testLoader, 'model1.pt', correctSamples, correctLabels, incorrectSamples)
  # scheduler.step(model1.test_losses[-1])
  for param_group in optimizer.param_groups:
    print('Learning Rate = {a} for EPOCH {e}'.format(a = round(param_group['lr'],5), e=epoch+1))
    learningRates.append(param_group['lr'])
  